In [ ]:
# rusty botパッケージのインストール（大幅修正中のため、Bybitのバックテストのみ動作確認完了）
! pip install -i https://test.pypi.org/simple/ rbot==0.1.6
! pip install polars_talib

In [ ]:
import rbot
rbot.mount_google_drive()   # google colabの場合GoogleDriveにダウンロードデータを保存します。

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import polars_talib as plta
import polars as pl
import numpy as np
import json

from rbot import Bybit, BybitConfig, Runner

In [ ]:
class PureRichmanAgent:
    def __init__(self, order_size: float=0.01, timebar_interval_sec: int=4 * 60 * 60, atr_multiple:float = 0.5, atr_bars: int=24):
        # agentの設定
        self.setting = {
            "order_size": order_size,
            "timebar_interval_sec": timebar_interval_sec,
            "atr_multiple": atr_multiple,
            "atr_bars": atr_bars
        }

        # タイムバーをagent内で保持しておくためのデータフレーム
        self.df_timebar = pl.DataFrame(schema={
            "timestamp": pl.Datetime,             # Datetime型へ変更しました。
            "open": pl.Float64,
            "high": pl.Float64,
            "low": pl.Float64,
            "close": pl.Float64,
            "volume": pl.Float64,
            "count": pl.UInt32,
        })

    def on_init(self, session):
        # on_clock呼び出し間隔を設定
        session.clock_interval_sec = self.setting["timebar_interval_sec"]

    def on_clock(self, session, clock):
        # rbotから最新のタイムバーをデータフレーム(1行)として取得
        _df_timebar = session.ohlcv(self.setting["timebar_interval_sec"], 1)

        if _df_timebar.shape[0] > 1:
            session.print("タイムバーを１本指定したので１本以下なはず（データがない場合は０になる可能性あり）。１本以上という条件のこのプリント文が表示されたらバグです")
            return

        if _df_timebar.shape[0] > 0:
            if _df_timebar["timestamp"][-1] == clock:
                # ウォームアップ中の呼び出しなので何もしない
                session.print("ウォームアップ中はよびだされないようにFix済み。これが表示されたらバグです")
                return
            else:
                # バックテスト中でかつ直前のタイムバーが生成されているので、ローカルのタイムバー保持用データフレームに追加
                self.df_timebar = self.df_timebar[-12:].vstack(_df_timebar[-1]).sort("timestamp", descending = False)
        else:
            # 直前のタイムバー期間でトレードがなく、直前のタイムバーが生成されていない
            if self.df_timebar.shape[0] > 0:
                # 直前のタイムバーのクローズ価格が取得できる場合は、価格変化なしのタイムバーを作成してローカルのデータフレームに追加
                _close = self.df_timebar["close"][-1]
                _df_timebar = pl.DataFrame({
                    "timestamp": clock,
                    "open": _close,
                    "high": _close,
                    "low": _close,
                    "close": _close,
                    "volume": 0.0,
                    "count": np.uint32(0)
                })
                self.df_timebar = self.df_timebar.vstack(_df_timebar)
            else:
                # 直前のクローズ価格が取得できないので、何もしない
                raise Exception("No past timebar data & No new timebar data")

        # ATRの計算
        _df_timebar_with_indicators = self.df_timebar.with_columns(
            plta.atr(pl.col("high"), pl.col("low"), pl.col("close"), self.setting["atr_bars"]).alias("atr"),
            (np.log(pl.col("close")) - np.log(pl.col("close").shift(1))).alias("lr")
        )
        _atr = _df_timebar_with_indicators["atr"][-1]
        _close = _df_timebar_with_indicators["close"][-1]
        _price_unit = 0.1

        if np.isnan(_atr) == False:
            # ATRが計算できたので、注文を出す

            # 残っている指値注文を全てキャンセル
            for _order in session.buy_orders:
                session.cancel_order(_order.order_id)
                session.print(f"on_clock ({clock}): cancel BUY LIMIT order: {_order}")    # session.printをつかうとプログレスバーと同居できます。
            for _order in session.sell_orders:
                session.cancel_order(_order.order_id)
                session.print(f"on_clock ({clock}): cancel SELL LIMIT order: {_order}")

            if session.position <= 0.0:
                # ショートポジションを取っているか、ポジションがないので買い指値を入れる
                _price = ((_close - self.setting["atr_multiple"] * _atr) // _price_unit) * _price_unit
                _amount = self.setting["order_size"] if session.position == 0.0 else abs(session.position)

                session.print(f"on_clock ({clock}): BUY LIMIT ORDER {_price}, {_amount}")
                session.limit_order("Buy", _price, _amount)
            if session.position >= 0.0:
                # ロングポジションを取っているか、ポジションがないので売り指値を入れる
                _price = ((_close + self.setting["atr_multiple"] * _atr) // _price_unit) * _price_unit
                _amount = self.setting["order_size"] if session.position == 0.0 else abs(session.position)

                session.print(f"on_clock ({clock}): SELL LIMIT ORDER {_price}, {_amount}")
                session.limit_order("Sell", _price, _amount)

    def on_update(self, session, update):
        # print(f"on_update: {update}")
        pass

In [ ]:

exchange = Bybit(production=True)
config = BybitConfig.BTCUSDT
market = exchange.open_market(config)
#market.download_archive(
#    ndays=365,        # specify from past days
#    force=True,    # if false, the cache data will be used.
#    verbose=True    # verbose to print download progress.
#)

# market.expire_unfix_data(force=True)    # アーカイブにはUnfixデータはないので削除。今回からアーカイブの範囲だけのバックテストへ変更となりました。

In [ ]:
# とりあえず20日分のダウンロード
market.download_archive(
    ndays=20,        # specify from past days
    force=False,    # if false, the cache data will be used.
    verbose=True    # verbose to print download progress.
)

In [ ]:
agent = PureRichmanAgent(0.01, 4 * 60 * 60, 0.5, 12)
runner = Runner()

session = runner.back_test(
                exchange=exchange,
                market=market,
                agent=agent,
                start_time=0,
                end_time=0,
                verbose=True    # Trueにしないとprintもいまのところ非表示になります。
            )

In [ ]:
orders = session.log.orders
print(f"トータルリターン (手数料込) : {orders['total_profit'].sum()}")
print(f"トータルリターン (手数料抜き) : {orders['profit'].sum()}")